In [4]:
import cv2
import numpy as np
import tensorflow as tf
IMG_WIDTH, IMG_HEIGHT = 150, 150

# Load the saved model
model = tf.keras.models.load_model(r'c:\Users\Dhruv\Desktop\capestone\MODELS\pose_classification_model_new_added_layer_MobileNetV2.h5')

# Function to preprocess the frame for prediction
def preprocess_frame(frame, img_width, img_height):
    resized_frame = cv2.resize(frame, (img_width, img_height))
    preprocessed_frame = np.expand_dims(resized_frame, axis=0)
    preprocessed_frame = preprocessed_frame / 255.0
    return preprocessed_frame

# Open the video file
video_path = (r'dataset/videos/icb.mp4')  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of correct, partially correct, and incorrect frames
total_frames = 0
correct_frames = 0
partially_correct_frames = 0
incorrect_frames = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for prediction
    preprocessed_frame = preprocess_frame(frame, IMG_WIDTH, IMG_HEIGHT)

    # Make predictions on the frame
    predictions = model.predict(preprocessed_frame)
    predicted_class = np.argmax(predictions, axis=1)

    # Update counters based on the predicted class
    total_frames += 1
    if predicted_class[0] == 0:  # Correct class index
        correct_frames += 1
    elif predicted_class[0] == 3:  # Partially Correct class index
        partially_correct_frames += 1
    elif predicted_class[0] == 1:  # Incorrect class index
        incorrect_frames += 1

    # Get the class label as a string
    class_labels = ["Correct", "Incorrect", "none", "Partially Correct"]
    predicted_class_label = class_labels[predicted_class[0]]

    # Display the predicted class on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(predicted_class_label, font, font_scale, font_thickness)[0]
    text_position = (10, 30)
    text_background_position = (10, 10)
    text_background_end_position = (10 + text_size[0] + 10, 10 + text_size[1] + 10)
    cv2.rectangle(frame, text_background_position, text_background_end_position, (0, 0, 255), cv2.FILLED)
    cv2.putText(frame, predicted_class_label, text_position, font, font_scale, (255, 255, 255), font_thickness)

    # Display the frame
    cv2.imshow('Pose Classification', frame)

    # Exit the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate the scores
step = 100 / total_frames
score1 = step * correct_frames
score2 = step / 2 * partially_correct_frames
total_score = score1 + score2

print(f"Total Frames: {total_frames}")
print(f"Correct Frames: {correct_frames}")
print(f"Partially Correct Frames: {partially_correct_frames}")
print(f"Incorrect Frames: {incorrect_frames}")
print(f"Total Score: {total_score:.2f}")

# Release the video capture and close the display window
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 32ms/step
Total Frames: 378
Correct Frames: 323
Partially Correct Frames: 48
Incorrect Frames: 7
Total Score: 91.80
